In [1]:
import subprocess
import os
import shutil
from pathlib import Path
import sys
from processing import utils as p_utils
import contextlib
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

root = Path.cwd().parent
library = root.parent / "reward-surfaces"
stash = root / "stash"
scripts = library / "scripts"

run_suffix = "_test"
env_name = "CartPole-v1"
agent_name = "SB3_ON"
run_id = "cartpole" + run_suffix

In [16]:
# Train RL agent
script_name = "train_agent"
args = {
    "save_dir": stash / run_id / "agent",
    "agent_name": agent_name,
    "env": env_name,
    "device": device,
    "hyperparameters": '{"ALGO": "PPO", "n_timesteps":50000}',
}
kwargs = {
    "--save_freq": str(10000),
}
p_utils.execute(name=script_name, args=args, kwargs=kwargs, cwd=library)

/home/mattis/.local/share/virtualenvs/reward_landscapes-JbenRfY6/lib/python3.8/site-packages/stable_baselines3/__init__.py
False
Default hyperparameters for environment (ones being tuned will be overridden):
OrderedDict([('batch_size', 256),
             ('clip_range', 'lin_0.2'),
             ('ent_coef', 0.0),
             ('gae_lambda', 0.8),
             ('gamma', 0.98),
             ('learning_rate', 'lin_0.001'),
             ('n_envs', 8),
             ('n_epochs', 20),
             ('n_steps', 32),
             ('n_timesteps', 50000),
             ('policy', 'MlpPolicy')])
Using 8 environments
cpu
Log path: /home/mattis/git/sem5/reward_landscapes/stash/cartpole_test/agent/ppo/CartPole-v1_5
saved checkpoint 0010000
Eval num_timesteps=10000, episode_reward=126.30 +/- 28.32
Episode length: 126.30 +/- 28.32
New best mean reward!
saved checkpoint 0020000
Eval num_timesteps=20000, episode_reward=269.36 +/- 48.96
Episode length: 269.36 +/- 48.96
New best mean reward!
saved checkpoint 

In [3]:
# Define gradient eval jobs
script_name = "generate_eval_jobs"
args = {
    "train_dir": stash / run_id / "agent",
    "out_dir": stash / run_id / "gradient",
}
kwargs = {
    "--batch-grad": None,
    "--num-steps": str(1000000),
    "--device": device,
}
shutil.rmtree(stash / run_id / "gradient")
p_utils.execute(name=script_name, args=args, kwargs=kwargs, cwd=library)

# Evaluate gradient direction
script_name = "run_jobs_multiproc"
args = {
    "job_dir": stash / run_id / "gradient" / "jobs.sh",
}
kwargs = {"--num-cpus": str(7)}
p_utils.execute(name=script_name, args=args, kwargs=kwargs, cwd=library)

/home/mattis/.local/share/virtualenvs/reward_landscapes-JbenRfY6/lib/python3.8/site-packages/stable_baselines3/__init__.py
['0010000', '0020000', '0030000', '0040000', '0050000']


  0%|          | 0/5 [00:00<?, ?it/s]

/home/mattis/.local/share/virtualenvs/reward_landscapes-JbenRfY6/lib/python3.8/site-packages/stable_baselines3/__init__.py
/home/mattis/.local/share/virtualenvs/reward_landscapes-JbenRfY6/lib/python3.8/site-packages/stable_baselines3/__init__.py
/home/mattis/.local/share/virtualenvs/reward_landscapes-JbenRfY6/lib/python3.8/site-packages/stable_baselines3/__init__.py
/home/mattis/.local/share/virtualenvs/reward_landscapes-JbenRfY6/lib/python3.8/site-packages/stable_baselines3/__init__.py
Default hyperparameters for environment (ones being tuned will be overridden):
OrderedDict([('batch_size', 256),
             ('clip_range', 'lin_0.2'),
             ('ent_coef', 0.0),
             ('gae_lambda', 0.8),
             ('gamma', 0.98),
             ('learning_rate', 'lin_0.001'),
             ('n_envs', 8),
             ('n_epochs', 20),
             ('n_steps', 32),
             ('n_timesteps', 50000),
             ('policy', 'MlpPolicy')])
Using 8 environments
Default hyperparameters for 

/home/mattis/git/sem5/reward-surfaces/reward_surfaces/algorithms/eval_policy_hess.py:281: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  torch.squeeze(torch.tensor(test_states[0][0:2], device=device), dim=1),
/home/mattis/git/sem5/reward-surfaces/reward_surfaces/algorithms/eval_policy_hess.py:281: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  torch.squeeze(torch.tensor(test_states[0][0:2], device=device), dim=1),


computing batch grad mag


/home/mattis/git/sem5/reward-surfaces/reward_surfaces/algorithms/eval_policy_hess.py:281: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  torch.squeeze(torch.tensor(test_states[0][0:2], device=device), dim=1),
/home/mattis/git/sem5/reward-surfaces/reward_surfaces/algorithms/eval_policy_hess.py:281: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  torch.squeeze(torch.tensor(test_states[0][0:2], device=device), dim=1),
/home/mattis/git/sem5/reward-surfaces/reward_surfaces/algorithms/eval_policy_hess.py:281: UserWarning: Creating a tensor from a list of numpy.ndarrays is ex

computing batch grad mag
computing batch grad mag
computing batch grad mag
Aborted!


Traceback (most recent call last):
  File "/home/mattis/.pyenv/versions/3.8.20/lib/python3.8/runpy.py", line 194, in _run_module_as_main
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/mattis/.pyenv/versions/3.8.20/lib/python3.8/runpy.py", line 194, in _run_module_as_main
  File "/home/mattis/.pyenv/versions/3.8.20/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,  File "/home/mattis/.pyenv/versions/3.8.20/lib/python3.8/runpy.py", line 194, in _run_module_as_main

  File "/home/mattis/.pyenv/versions/3.8.20/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/mattis/git/sem5/reward-surfaces/scripts/run_jobs_multiproc.py", line 16, in <module>
    return _run_code(code, main_globals, None,
  File "/home/mattis/.pyenv/versions/3.8.20/lib/python3.8/runpy.py", line 87, in _run_code
    return _run_code(code, main_globals, None,
  

In [ ]:
# Define plane jobs
script_name = "generate_plane_jobs"
args = {
    "agent_dir": stash / run_id / "agent" / "best",
    "out_dir": stash / run_id / "surface",
}
kwargs = {
    "--grid-size": "31",
    "--magnitude": "1.0",
    "--num-steps": str(200000),
}
# shutil.rmtree(stash / run_id / "surface")
p_utils.execute(name=script_name, args=args, kwargs=kwargs, cwd=library)

# Evaluate plane directions
script_name = "run_jobs_multiproc"
args = {
    "job_dir": stash / run_id / "surface" / "jobs.sh",
}
kwargs = {"--num-cpus": str(7)}
p_utils.execute(name=script_name, args=args, kwargs=kwargs, cwd=library)

# Copy surface results to csv
script_name = "job_results_to_csv"
args = {
    "out_dir": stash / run_id / "surface",
}
kwargs = {}
p_utils.execute(name=script_name, args=args, kwargs=kwargs, cwd=library)

/home/mattis/.local/share/virtualenvs/reward_landscapes-JbenRfY6/lib/python3.8/site-packages/stable_baselines3/__init__.py
Default hyperparameters for environment (ones being tuned will be overridden):
OrderedDict([('batch_size', 256),
             ('clip_range', 'lin_0.2'),
             ('ent_coef', 0.0),
             ('gae_lambda', 0.8),
             ('gamma', 0.98),
             ('learning_rate', 'lin_0.001'),
             ('n_envs', 8),
             ('n_epochs', 20),
             ('n_steps', 32),
             ('n_timesteps', 50000),
             ('policy', 'MlpPolicy')])
Using 8 environments
cpu
Log path: /home/mattis/git/sem5/reward_landscapes/stash/cartpole_test/surface/ppo/CartPole-v1_4
/home/mattis/.local/share/virtualenvs/reward_landscapes-JbenRfY6/lib/python3.8/site-packages/stable_baselines3/__init__.py


In [ ]:
# Plot reward surface
script_name = "plot_plane"
args = {
    "in_dir": stash / run_id / "surface" / "results.csv",
}
kwargs = {
    "--outname": stash / run_id / "surface",
    "--env_name": "CartPole-v1",
}
p_utils.execute(name=script_name, args=args, kwargs=kwargs, cwd=library)

/home/mattis/.local/share/virtualenvs/reward_landscapes-JbenRfY6/lib/python3.8/site-packages/stable_baselines3/__init__.py


['python3',
 'scripts/plot_plane.py',
 './runs/cartpole_surface/results.csv--outname',
 './runs/cartpole--env_name',
 'CartPole-v1']

In [ ]:
# Evaluate random directions
[
    "python3",
    "scripts/generate_plane_jobs.py",
    "--grid-size=31",
    "--magnitude=1.0",
    "--num-steps",
    str(200000),
    "./runs/cartpole_checkpoints/best/",
    "./runs/cartpole_surface/",
]
[
    "python3",
    "scripts/run_jobs_multiproc.py",
    "--num-cpus",
    str(8),
    "./runs/cartpole_surface/jobs.sh",
]
["python3", "scripts/job_results_to_csv.py", "./runs/cartpole_surface/"]
# Plot plane (random directions)
[
    "python3",
    "scripts/plot_plane.py",
    "./runs/cartpole_surface/results.csv" "--outname",
    "./runs/cartpole" "--env_name",
    "CartPole-v1",
]
[
    "python",
    "scripts/eval_line_segment.py",
    "./runs/cartpole_checkpoints/",
    "./runs/eval_grad/cartpole/results",
    "./runs/eval_line/cartpole/" "--num-episodes",
    str(100),
    "--length",
    str(20),
    "--max-magnitude",
    str(0.4),
    "--scale-dir",
]
[
    "python",
    "scripts/run_jobs_multiproc.py",
    "--num-cpus=8",
    "./runs/eval_line/cartpole/jobs.sh",
]
["python", "scripts/job_results_to_csv.py", "./runs/eval_line/cartpole/"]
[
    "python",
    "scripts/plot_eval_line_segement.py",
    "./runs/eval_line/cartpole/results.csv" "--outname",
    "cartpole__lines.png",
]